In [91]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [182]:
crdf = complete_reports_df
cmdf = complete_measurements_df

categories = list(crdf.category.sort_values().unique()) # ['A', 'B', 'C', 'D']
selected_year = 2017

selected_ind = 'ind16'
default_cat = 'A'

ydf = cmdf[cmdf.year == selected_year]

data = []

for i, category in enumerate(categories):
    fdf1 = ydf[ydf.category == category]
    fdf = fdf1[np.isfinite(fdf1[selected_ind])].sort_values(selected_ind)
    visible = True if i == 0 else False
    data.append(go.Bar(
        x=list(fdf.code),
        y=list(fdf[selected_ind]),
        text= [f'{code}<br>' + str(val) + get_ind_unit(selected_ind) for code,val in zip(list(fdf.code),list(fdf[selected_ind]))],
        opacity=0.8,  
        textposition='auto',
        hoverinfo='text',
        visible=visible,
    ))

def get_visible_list(category):
    category_map = dict(A=0, B=1, C=2, D=3)
    base_array = [False for i in range(4)]
    base_array[category_map[category]] = True
    return base_array

def make_x_axis(category):
    return dict(
        title=f'EPSAs Categoría {category}',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

def make_y_axis():
    return dict(
        title=f'Presión del servicio de AP ({get_ind_unit(selected_ind)})',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

updatemenus = [dict(
    type= 'buttons',
    showactive=True,
    active=0,
    xanchor='left',
    yanchor='top',
    direction='right',
    x= 0.25,
    y= -0.3,
    buttons = [dict(
        label= f'Categoría {cat}',
        method='update', # modify both data and layout
        args = [
            {'visible': get_visible_list(cat)}, # data modification
            dict(
                title= f'Presión del Servicio de Agua Potable - Categoría {cat}',
                xaxis= make_x_axis(cat),
            ),
        ],
    ) for cat in categories]
)]

layout = go.Layout(
    title= 'Presión del Servicio de Agua Potable - Categoría A',
    xaxis= make_x_axis('A'),
    yaxis= make_y_axis(),
    hovermode='closest',
    updatemenus=updatemenus,
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [183]:
import plotly.plotly as py
py.iplot(fig, filename='pressure_2017')